In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig
import torch
from transformers import BitsAndBytesConfig
import os


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
base_model_name_or_path = "../checkpoint/Taiwan-LLM-7B-v2.0-chat"

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [6]:
peft_model_id = "../checkpoint/finetune_full_module_warmup/checkpoint-200"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(base_model_name_or_path, quantization_config=quantization_config, torch_dtype=torch.bfloat16)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
model = model.to("cuda")
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CUDA extension not installed.
CUDA extension not installed.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): ModuleDict(
         

In [7]:
def get_prompt(instruction: str) -> str:
    '''Format the instruction as a prompt for LLM.'''
    return f"你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: {instruction} ASSISTANT:"


In [8]:
generation_config = GenerationConfig(
                repetition_penalty=1.1,
                max_new_tokens=1024,
                temperature=0.9,
                top_p=0.95,
                top_k=40,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=True,
                use_cache=True,
                return_dict_in_generate=True,
                output_attentions=False,
                output_hidden_states=False,
                output_scores=False,
            )

In [10]:
inputs = tokenizer(get_prompt("""翻譯成文言文：\n於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。
-> 帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。
文言文翻譯：\n靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。
-> 答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。
希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。\n這句話在古代怎麼說：
-> 以後幸長官留意，勿令如此。
第二年召迴朝廷，改任著作佐郎，直史館，改任左拾遺。\n翻譯成文言文：
->："""), return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), generation_config=generation_config)
    print(tokenizer.batch_decode(outputs['sequences'].cpu().numpy(), skip_special_tokens=True)[0])

你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。USER: 翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。
-> 帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。
文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。
-> 答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。
希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說：
-> 以後幸長官留意，勿令如此。
第二年召迴朝廷，改任著作佐郎，直史館，改任左拾遺。
翻譯成文言文：
->： ASSISTANT: 次赴朝，除著作佐郎，直史館，復升左拾遺。
